<a href="https://colab.research.google.com/github/sumukh12/Flip-robo/blob/main/CNN_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 4.6 MB/s 


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
print(tf.__version__)

2.6.0


In [6]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [11]:
#Build model
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=64,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D( 
          filters=hp.Int('conv_2_filters',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense( 
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
    ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
               loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")


In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.8488333225250244

Best val_accuracy So Far: 0.909500002861023
Total elapsed time: 00h 03m 34s
INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1769520   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,845,082
Trainable params: 1,845,082
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1346 - accuracy: 0.9493 - val_loss: 0.2607 - val_accuracy: 0.9157
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0963 - accuracy: 0.9645 - val_loss: 0.2973 - val_accuracy: 0.9152
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0679 - accuracy: 0.9750 - val_loss: 0.3263 - val_accuracy: 0.9062
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0490 - accuracy: 0.9817 - val_loss: 0.3855 - val_accuracy: 0.9155
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0363 - accuracy: 0.9871 - val_loss: 0.4469 - val_accuracy: 0.9133
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0279 - accuracy: 0.9897 - val_loss: 0.4938 - val_accuracy: 0.9132
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0223 - accuracy: 0.9921 - val_loss: 0.5688 - val_accuracy